In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

/home/rupsas/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/stortinget_dataset.csv.gz')

In [3]:
df

,sakid,tittel,emneord,url,sammendrag,vedtak,text,html,tilrading
0,27712,"Forslag fra Gunnar Skaug, Carl I. Hagen, Einar...","['Massemedier', 'Menneskerettigheter', 'Grunnl...",https://www.stortinget.no/no/Saker-og-publikas...,Til Stortinget 1.\tInnledning Dokument nr. 12:...,I Dokument nr. 12:16 (1999-2000) - forslag fra...,Innst. S. nr. 270 (2003-2004) - stortinget.no ...,"\n<!DOCTYPE html>\n\n<html class=""no-js"" lang=...",NaN
1,27713,Forslag fra Gunnar Kvassheim og Lars Sponheim ...,"['Statsforvaltning', 'Grunnloven']",https://www.stortinget.no/no/Saker-og-publikas...,Til Stortinget 1.\tInnledning Dokument nr. 12:...,I Dokument nr. 12:16 (1999-2000) - forslag fra...,Innst. S. nr. 270 (2003-2004) - stortinget.no ...,"\n<!DOCTYPE html>\n\n<html class=""no-js"" lang=...",NaN
2,27805,Endring av Grunnloven § 100,"['Massemedier', 'Menneskerettigheter', 'Grunnl...",https://www.stortinget.no/no/Saker-og-publikas...,Til Stortinget 1.\tInnledning Dokument nr. 12:...,I Dokument nr. 12:16 (1999-2000) - forslag fra...,Innst. S. nr. 270 (2003-2004) - stortinget.no ...,"\n<!DOCTYPE html>\n\n<html class=""no-js"" lang=...",NaN
3,27709,"Forslag fra Thomas Chr. Wyller, vedtatt til fr...","['Internasjonalt samarbeid', 'Statsforfatning'...",https://www.stortinget.no/no/Saker-og-publikas...,Til Stortinget Sammendrag I Dokument nr. 12 (1...,"I Dokument nr. 12:13 (1999-2000), forslag fra\...",Innst. S. nr. 190 (2003-2004) - stortinget.no ...,"\n<!DOCTYPE html>\n\n<html class=""no-js"" lang=...",NaN
4,27710,Forslag fra Erna Solberg om ny § 93 a i Grunnl...,"['Statsforfatning', 'Eu/eøs', 'Grunnloven']",https://www.stortinget.no/no/Saker-og-publikas...,Til Stortinget Sammendrag I Dokument nr. 12 (1...,"I Dokument nr. 12:13 (1999-2000), forslag fra\...",Innst. S. nr. 190 (2003-2004) - stortinget.no ...,"\n<!DOCTYPE html>\n\n<html class=""no-js"" lang=...",NaN
...,...,...,...,...,...,...,...,...,...
17216,99388,Grunnlovsforslag fra Kathy Lie og Grete Wold o...,"['Forskning', 'Grunnloven']",NaN,NaN,NaN,NaN,NaN,NaN
17217,99311,Grunnlovsforslag fra Michael Tetzschner om § 9...,"['Rettsvesen', 'Domstoler', 'Skatter', 'Merver...",NaN,NaN,NaN,NaN,NaN,NaN
17218,99312,Grunnlovsforslag fra Michael Tetzschner og Mor...,"['Sivilrett', 'Statsforvaltning', 'Grunnloven']",NaN,NaN,NaN,NaN,NaN,NaN
17219,100004,Grunnlovsforslag fremsatt på det 167. og 168. ...,['Grunnloven'],NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['sakid', 'tittel', 'emneord', 'url', 'sammendrag', 'vedtak', 'text',
       'html', 'tilrading'],
      dtype='object')

In [5]:
print('Sammendrag:',df['sammendrag'].isna().sum())

print('Vedtak:',df['vedtak'].isna().sum())

print('Text:',df['text'].isna().sum())

Sammendrag: 4937
Vedtak: 10109
Text: 3951


In [15]:
print('No Sammendrag & Vedtak, Only Text :',df[df[['sammendrag', 'vedtak']].isna().all(axis=1) & df['text'].notna()].shape[0])
print('No Text & Vedtak, Only Sammendrag :',df[df[['text', 'vedtak']].isna().all(axis=1) & df['sammendrag'].notna()].shape[0])
print('No Text & Sammendrag, Only Vedtak :',df[df[['text', 'sammendrag']].isna().all(axis=1) & df['vedtak'].notna()].shape[0])
print('No Sammendrag, Vedtak, Text :',df[df[['sammendrag', 'vedtak', 'text']].isna().all(axis=1)].shape[0])
print('No Sammendrag, Only Text & Vedtak :',df[df[['text', 'vedtak']].notna().all(axis=1) & df['text'].isna()].shape[0])
print('No Vedtak, Only Text & Sammendrag :',df[df[['text', 'sammendrag']].notna().all(axis=1) & df['vedtak'].isna()].shape[0])
print('No Text, Only Vedtak & Sammendrag :',df[df[['vedtak', 'sammendrag']].notna().all(axis=1) & df['text'].isna()].shape[0])


print('Total:', df.shape[0])

No Sammendrag & Vedtak, Only Text : 1
No Text & Vedtak, Only Sammendrag : 0
No Text & Sammendrag, Only Vedtak : 0
No Sammendrag, Vedtak, Text : 3951
No Sammendrag, Only Text & Vedtak : 0
No Vedtak, Only Text & Sammendrag : 6157
No Text, Only Vedtak & Sammendrag : 0
Total: 17221


In [16]:
# Import model
tokenizer = AutoTokenizer.from_pretrained(
    "ltg/norbert4-large"
)

In [19]:
text = df.loc[0, 'vedtak']
text

'I Dokument nr. 12:16 (1999-2000) - forslag fra Gunnar Skaug, Carl I. Hagen, Einar Steensnæs, Jan Petersen, Kristin Halvorsen og Lars Sponheim om endring av Grunnloven § 100 (Ytringsfrihet) - bifalles som fø\xadlger: Første ledd: Alternativ 1 bifalles. Annet ledd: Hovedalternativ 16 bifalles. Til annet ledd annet punktum bifalles dessuten hovedalternativ 2 annet ledd tredje punktum alternativ 3. Tredje ledd: Hovedalternativ 16 alternativ 1 bifalles. Til tredje ledd første punktum bifalles dessuten fjerde ledd første punktum alternativ 5. Til tredje ledd annet punktum bifalles dessuten fjerde ledd annet punktum alternativ 5. Til tredje ledd tredje punktum bifalles dessuten fjerde ledd tredje punktum alternativ 1 og presisert som fjerde ledd annet punktum alternativ 3. Fjerde ledd: Til første punktum bifalles alternativ 8. Til annet punktum bifalles alternativ 6. Til tredje punktum bifalles alternativ 2. Femte ledd: Til første punktum bifalles alternativ 1. Til annet punktum bifalles alt

In [20]:
tokenizer.tokenize(text)

['I',
 'ĠDokument',
 'Ġnr',
 '.',
 'Ġ',
 '1',
 '2',
 ':',
 '1',
 '6',
 'Ġ(',
 '1',
 '9',
 '9',
 '9',
 '-',
 '2',
 '0',
 '0',
 '0',
 ')',
 'Ġ-',
 'Ġforslag',
 'Ġfra',
 'ĠGunnar',
 'ĠSka',
 'ug',
 ',',
 'ĠCarl',
 'ĠI',
 '.',
 'ĠHagen',
 ',',
 'ĠEinar',
 'ĠSte',
 'ens',
 'nÃ¦s',
 ',',
 'ĠJan',
 'ĠPetersen',
 ',',
 'ĠKristin',
 'ĠHalvorsen',
 'Ġog',
 'ĠLars',
 'ĠSp',
 'on',
 'heim',
 'Ġom',
 'Ġendring',
 'Ġav',
 'ĠGrunn',
 'loven',
 'ĠÂ§',
 'Ġ',
 '1',
 '0',
 '0',
 'Ġ(',
 'Y',
 't',
 'ring',
 'sfrihet',
 ')',
 'Ġ-',
 'Ġbif',
 'al',
 'les',
 'Ġsom',
 'ĠfÃ¸',
 'ÂŃ',
 'l',
 'ger',
 ':',
 'ĠFÃ¸rste',
 'Ġledd',
 ':',
 'ĠAlternativ',
 'Ġ',
 '1',
 'Ġbif',
 'al',
 'les',
 '.',
 'ĠAnnet',
 'Ġledd',
 ':',
 'ĠHoved',
 'alternativ',
 'Ġ',
 '1',
 '6',
 'Ġbif',
 'al',
 'les',
 '.',
 'ĠTil',
 'Ġannet',
 'Ġledd',
 'Ġannet',
 'Ġpunktum',
 'Ġbif',
 'al',
 'les',
 'Ġdessuten',
 'Ġhoved',
 'alternativ',
 'Ġ',
 '2',
 'Ġannet',
 'Ġledd',
 'Ġtredje',
 'Ġpunktum',
 'Ġalternativ',
 'Ġ',
 '3',
 '.',
 'ĠTredje',
 'Ġl

In [24]:
df[df['emneord'].map(len) == 2]

,sakid,tittel,emneord,url,sammendrag,vedtak,text,html,tilrading
203,19669,Melding for året 1999 fra Sivilombudsmannen (S...,[],https://www.stortinget.no/no/Saker-og-publikas...,Til Stortinget 1. Innledning Stortingets ombud...,Dokument nr. 4 (1999-2000) - melding for året ...,Innst. S. nr. 208 (1999-2000) - stortinget.no ...,"\n<!DOCTYPE html>\n\n<html class=""no-js"" lang=...",NaN
223,18944,Forslag fra stortingsrepresentantene Gunnar Sk...,[],NaN,NaN,NaN,NaN,NaN,NaN
276,17980,Lov om erverv av og drift på mineralressurser ...,[],NaN,NaN,NaN,NaN,NaN,NaN
281,18127,Lov om endringer i lov 9. februar 1973 nr. 6 o...,[],NaN,NaN,NaN,NaN,NaN,NaN
385,19476,Brev av 23. februar 2000 fra Nærings- og hande...,[],https://www.stortinget.no/no/Saker-og-publikas...,Til Stortinget 1. Innledning og sammendrag 1.1...,St.meld. nr. 41 (1998-1999) - om elektronisk\n...,Innst. S. nr. 127 (1999-2000) - stortinget.no ...,"\n<!DOCTYPE html>\n\n<html class=""no-js"" lang=...",NaN
...,...,...,...,...,...,...,...,...,...
17041,101169,Skatter og avgifter 2025,[],Innstilling fra finanskomiteen om bevilgninger...,NaN,"﻿\r\n<!DOCTYPE html>\n\n<html class=""no-js"" la...",https://www.stortinget.no/no/Saker-og-publikas...,NaN,﻿ Innst. 5 S (2024-2025) - stortinget.no Du b...
17042,101171,Endring av Prop. 1 S (2024-2025) Statsbudsjett...,[],Innstilling fra finanskomiteen om bevilgninger...,NaN,"﻿\r\n<!DOCTYPE html>\n\n<html class=""no-js"" la...",https://www.stortinget.no/no/Saker-og-publikas...,NaN,﻿ Innst. 5 S (2024-2025) - stortinget.no Du b...
17054,101150,Endring av Prop. 1 S (2024-2025) Statsbudsjett...,[],Innstilling fra utenriks- og forsvarskomiteen ...,NaN,"﻿\r\n<!DOCTYPE html>\n\n<html class=""no-js"" la...",https://www.stortinget.no/no/Saker-og-publikas...,NaN,﻿ Innst. 17 S (2024-2025) - stortinget.no Du ...
17101,101135,"Innst. 3 S Skatte-, avgifts- og tollinntekter ...",[],Innstilling fra finanskomiteen om Skatter og a...,NaN,"﻿\r\n<!DOCTYPE html>\n\n<html class=""no-js"" la...",https://www.stortinget.no/no/Saker-og-publikas...,NaN,﻿ Innst. 3 S (2024-2025) - stortinget.no Du b...
